<a href="https://colab.research.google.com/github/gmehra123/data_science_projs/blob/main/Optimzation_practice_problems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 35.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from pulp import *

### Investment problem
1. Total Investmenr=100K
2. 3 options
  * Low risk fund returns 5%
  * Med Risk fund returns 10%
  * High Risk fund returns 20%
Constraints
* max of 10% of total in high risk fund
* max of 20% with medium risk

objective is to maximize returns


In [3]:
model=LpProblem('Investment',LpMaximize)

In [4]:
# defining the Decision variable
returns=[0.05,0.10,0.20]
funds=['A','B','C']
x=LpVariable.dicts('fund',[i for i in funds],lowBound=0,cat='Continous')
x
x['A']

fund_A

In [5]:
# objective function
model+=0.05*x['A']+0.10*x['B']+0.20*x['C']
# Now adding the constraints
model+=x['C']<=10000
model+=x['B']<=20000
model+=x['A']+x['B']+x['C']==100000

In [6]:
model.solve()

1

In [7]:
for v in model.variables():
  print(v, '= ',v.varValue)

fund_A =  70000.0
fund_B =  20000.0
fund_C =  10000.0


### Simple Construction problem
* Maximize revenue by choosing projects
* There are 5 projects so 5 Binary decision varibales
* objective function= P['A']* 500 + P['B'] *4000...etc
* Constraints are the number of teams and the company has only 5 teams
 * A req 1 team
 * B requires 3 teams
 * c--> 2
 * d--> 1
 * e--> 5

In [8]:
projs=['A','B','C','D','E']
x=LpVariable.dicts('choose_proj',[i for i in projs],cat='Binary')

In [9]:
model=LpProblem('Construction_Assignment', LpMaximize)
model+=x['A']*500+x['B']*4000+x['C']*3000+x['D']*2000+x['E']*2000

In [10]:
# Adding constraints
model+=1*x['A']+3*x['B']+2*x['C']+1*x['D']+5*x['E']<=5
model+=x['C']-x['A']<=0
model+=x['D']-x['A']<=0
model+=x['D']-x['C']<=0

In [11]:
model.solve()

1

In [12]:
for v in model.variables():
  print (v,'=',v.varValue)

choose_proj_A = 1.0
choose_proj_B = 0.0
choose_proj_C = 1.0
choose_proj_D = 1.0
choose_proj_E = 0.0


Soln is choose projs A,C,D

### Job Scheduling

In [13]:
jobs=['A','B','C','D','E','F']
days=['1','2','3']
x=LpVariable.dicts('job_schedule',[(i,j) for i in jobs for j in days],cat='Binary')
x

{('A', '1'): job_schedule_('A',_'1'),
 ('A', '2'): job_schedule_('A',_'2'),
 ('A', '3'): job_schedule_('A',_'3'),
 ('B', '1'): job_schedule_('B',_'1'),
 ('B', '2'): job_schedule_('B',_'2'),
 ('B', '3'): job_schedule_('B',_'3'),
 ('C', '1'): job_schedule_('C',_'1'),
 ('C', '2'): job_schedule_('C',_'2'),
 ('C', '3'): job_schedule_('C',_'3'),
 ('D', '1'): job_schedule_('D',_'1'),
 ('D', '2'): job_schedule_('D',_'2'),
 ('D', '3'): job_schedule_('D',_'3'),
 ('E', '1'): job_schedule_('E',_'1'),
 ('E', '2'): job_schedule_('E',_'2'),
 ('E', '3'): job_schedule_('E',_'3'),
 ('F', '1'): job_schedule_('F',_'1'),
 ('F', '2'): job_schedule_('F',_'2'),
 ('F', '3'): job_schedule_('F',_'3')}

In [14]:
model=LpProblem('Job_schedule',LpMaximize)

In [15]:
hours=[2,3,5,2,6,4]
rev=[200,500,300,100,1000,300]
rev=dict(zip(jobs,rev))
rev

{'A': 200, 'B': 500, 'C': 300, 'D': 100, 'E': 1000, 'F': 300}

In [16]:
#Objective Function
z=lpSum([x[(j,i)]*rev[j] for j in jobs for i in days ])
model+=z

In [17]:
z

200*job_schedule_('A',_'1') + 200*job_schedule_('A',_'2') + 200*job_schedule_('A',_'3') + 500*job_schedule_('B',_'1') + 500*job_schedule_('B',_'2') + 500*job_schedule_('B',_'3') + 300*job_schedule_('C',_'1') + 300*job_schedule_('C',_'2') + 300*job_schedule_('C',_'3') + 100*job_schedule_('D',_'1') + 100*job_schedule_('D',_'2') + 100*job_schedule_('D',_'3') + 1000*job_schedule_('E',_'1') + 1000*job_schedule_('E',_'2') + 1000*job_schedule_('E',_'3') + 300*job_schedule_('F',_'1') + 300*job_schedule_('F',_'2') + 300*job_schedule_('F',_'3') + 0

In [18]:
# Constraints
duration=[2,3,5,2,6,4]
dur=dict(zip(jobs,duration))
dur

{'A': 2, 'B': 3, 'C': 5, 'D': 2, 'E': 6, 'F': 4}

In [19]:
# 6 hours in a day constraint
for i in days:
  model+=lpSum([x[(j,i)]*dur[j] for j in jobs])<=6

In [20]:
# Every job can only be done once in the next three days
for j in jobs:
  model+=lpSum([x[(j,i)] for i in days])<=1

In [21]:
model.solve()

1

In [22]:
for v in model.variables():
  print(v, '=', v.varValue)

job_schedule_('A',_'1') = 0.0
job_schedule_('A',_'2') = 0.0
job_schedule_('A',_'3') = 1.0
job_schedule_('B',_'1') = 0.0
job_schedule_('B',_'2') = 0.0
job_schedule_('B',_'3') = 1.0
job_schedule_('C',_'1') = 0.0
job_schedule_('C',_'2') = 0.0
job_schedule_('C',_'3') = 0.0
job_schedule_('D',_'1') = 1.0
job_schedule_('D',_'2') = 0.0
job_schedule_('D',_'3') = 0.0
job_schedule_('E',_'1') = 0.0
job_schedule_('E',_'2') = 1.0
job_schedule_('E',_'3') = 0.0
job_schedule_('F',_'1') = 1.0
job_schedule_('F',_'2') = 0.0
job_schedule_('F',_'3') = 0.0


In [23]:
model.objective.value()

2100.0

### capacitive Plant problem
* You have 4 locations where you can build 2 types of plants (High Cap or low Cap)
* Each location or country has an associated demand that mudt be fulfilled.
* You need to decide what kind of plant you need to build at each location and you need to decide how much to produce at each location.
* The objective is to minimize both the total cost fixed and variable subject to the demand constraint

In [82]:
#Creating the demand data frame
countries=['USA','Germany','Japan','Brazil','India']
cap1=['Low_cap','High_cap']
dem=pd.DataFrame({'demand':[2719.6,84.1,1676.8,145.4,156.4]},index=countries,columns=['demand'])
dem

,demand
USA,2719.6
Germany,84.1
Japan,1676.8
Brazil,145.4
India,156.4


In [83]:
# Creating the Capacity data Frame
cap=pd.DataFrame({'Low_cap':[500,500,500,500,500],'High_cap':[1500,1500,1500,1500,1500]},index=countries)
cap

,Low_cap,High_cap
USA,500,1500
Germany,500,1500
Japan,500,1500
Brazil,500,1500
India,500,1500


In [84]:
# Creating the variable cost Data  frame
var_cost=pd.DataFrame({'USA':[6,13,20,12,12],'Germany':[13,6,14,14,13],'Japan':[20,14,3,21,10],'Brazil':[12,14,21,8,23],'India':[17,13,9,21,8]},index=countries)

In [85]:
# Creating the fixed cost data frame
fix_cost=pd.DataFrame({'Low_cap':[6500,4980,6230,3230,2110],'High_cap':[9500,7270,9100,4730,3080]},index=countries)
fix_cost

,Low_cap,High_cap
USA,6500,9500
Germany,4980,7270
Japan,6230,9100
Brazil,3230,4730
India,2110,3080


In [86]:
#Defining the LP problem
prob=LpProblem('Production_Plan',LpMinimize)

In [87]:
# Defining the Decision varibales
# Variable 1 is defining what type of plant to open at location i
plant_choice=LpVariable.dicts('plant_type_choice',[(i,j)for i in countries for j in cap],cat='Binary')
prod_num=LpVariable.dicts('Prod_numbers',[(i,j) for i in countries for j in countries],cat='Continous',lowBound=0)

In [88]:
# Defining the objective Function
prob+=lpSum([plant_choice[(i,j)]*fix_cost.loc[i,j] for i in countries for j in cap])+lpSum([prod_num[(i,j)]*var_cost.loc[i,j] for i in countries for j in countries])

In [89]:
# Defining the constraints
# Demand constraint
for j in countries:
  prob+=lpSum([prod_num[(i,j)] for i in countries])==dem.loc[j,'demand']

In [92]:
# Capacity constraint
for i in countries:
  prob+=lpSum([prod_num[(i,j)]for j in countries])<=lpSum(cap.loc[i,s]*plant_choice[(i,s)] for s in cap1)


In [93]:
prob.solve()

1

,Low_cap,High_cap
USA,500,1500
Germany,500,1500
Japan,500,1500
Brazil,500,1500
India,500,1500


### Simulating the output of a model
* You can simulate the constraints and values in the objective function and see how the optimal solution changes. You can generally add some noise to the values.
* This combines Monte Carlo simulation and optimization.
